In [1]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()  # データセットの読み込み

#print(diabetes.DESCR)

print(type(diabetes))

<class 'sklearn.utils._bunch.Bunch'>


In [2]:
import pandas as pd

df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)  # データフレーム型に格納、特徴量の名前を列に加える
print(df.head())

df['Progression'] = diabetes.target  # ターゲットデータ（教師データ）もデータフレームに格納

print(df.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019907 -0.017646  
1 -0.039493 -0.068332 -0.092204  
2 -0.002592  0.002861 -0.025930  
3  0.034309  0.022688 -0.009362  
4 -0.002592 -0.031988 -0.046641  
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torch import optim

data = diabetes.data
label = diabetes.target

print(type(data))  # ndarray型
print(type(label))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [4]:
train_data, test_data, train_label, test_label = train_test_split(  # trainデータとtestデータに分割
    data, label, test_size=0.2)
print("train_data size: {}".format(len(train_data)))
print("test_data size: {}".format(len(test_data)))
print("train_label size: {}".format(len(train_label)))
print("test_label size: {}".format(len(test_label)))

print(type(train_data))  # 型は変わらない

train_data size: 353
test_data size: 89
train_label size: 353
test_label size: 89
<class 'numpy.ndarray'>


In [5]:
# 型をテンソルに変える
train_x = torch.Tensor(train_data)
test_x = torch.Tensor(test_data)
train_y = torch.Tensor(train_label)
test_y = torch.Tensor(test_label)
print(type(train_x))

<class 'torch.Tensor'>


In [6]:
# データをテンソルのデータにセットする. 引数として 入力データ, 教師データ とすれば良い
train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)
print(type(train_dataset))

<class 'torch.utils.data.dataset.TensorDataset'>


In [7]:
# データローダーの際の細かい設定をする
train_batch = DataLoader(
    dataset=train_dataset,  # データセットの指定
    batch_size=5,  # バッチサイズの指定
    shuffle=True,  # シャッフルするかどうかの指定
    num_workers=2  # コア数
)
test_batch = DataLoader(
    dataset=test_dataset,
    batch_size=5,
    shuffle=False,
    num_workers=2
)

## for文で回してサイズを確認する
for data, label in train_batch:
    print("batch data size: {}".format(data.size()))  # バッチサイズが5, 特徴量（軸）が10つと分かる
    print("batch label size: {}".format(label.size()))  # バッチサイズはもちろん上と同じ5, 回帰値が入っている
    break

batch data size: torch.Size([5, 10])
batch label size: torch.Size([5])


In [8]:
# ニューラルネットワークを作成
class Net(nn.Module):
    def __init__(self, D_in, H, D_out):  # モデルの定義. 重み, バイアスの初期化やその際のシード値固定もいずれしよう！
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):  # 順伝播の際の動きをプログラムする
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [12]:
## パラメータ表示
# ハイパーパラメータ
D_in = 10  # 入力次元（diabetesデータは10次元）
H = 100  # 隠れ層次元
D_out = 1  # 出力次元（回帰問題なので1次元）
epoch = 100  # 学習回数
## デバイスの指定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## ネットワーク実行
net = Net(D_in, H, D_out).to(device)
print("Device: {}".format(device))

Device: cpu


In [13]:
# 使用する損失関数の定義（回帰なのでMSE: Mean Squared Error）
criterion = nn.MSELoss()
# 使用する最適化関数の定義
optimizer = optim.Adam(net.parameters())

In [14]:
## 学習に必要な空リストを作成
train_loss_list = []  # 学習損失
test_loss_list = []  # 評価損失

# 学習の実行
for I in range(epoch):
    # 学習の進行状況を表示
    print('--------')
    print("Epoch: {}/{}".format(I + 1, epoch))
    # 損失の初期化
    train_loss = 0  # 学習損失
    test_loss = 0  # 評価損失
    # 学習モードに設定
    net.train()
    # ミニバッチごとにデータをロードして学習
    for data, label in train_batch:
        data = data.to(device)
        label = label.to(device)
        # 勾配を初期化
        optimizer.zero_grad()
        # データを入力して予測値を計算
        y_pred = net(data)
        # 損失を計算
        loss = criterion(y_pred.squeeze(), label)
        # 勾配を計算
        loss.backward()
        # パラメータの更新
        optimizer.step()
        # ミニバッチごとの損失を蓄積
        train_loss += loss.item()
    # ミニバッチの平均の損失を計算
    batch_train_loss = train_loss / len(train_batch)

    # 評価モードに設定
    net.eval()
    # 評価時に自動微分をゼロにする
    with torch.no_grad():
        for data, label in test_batch:
            data = data.to(device)
            label = label.to(device)
            # データを入力して予測値を計算
            y_pred = net(data)
            # 損失を計算
            loss = criterion(y_pred.squeeze(), label)
            # ミニバッチごとの損失を備蓄
            test_loss += loss.item()
    # ミニバッチの平均の損失を計算
    batch_test_loss = test_loss / len(test_batch)

    # エポックごとに損失を表示
    print("Train_Loss: {:.4f}".format(batch_train_loss))
    print("Test_Loss: {:.4f}".format(batch_test_loss))

    # 損失をリスト化して保存
    train_loss_list.append(batch_train_loss)
    test_loss_list.append(batch_test_loss)

--------
Epoch: 1/100
Train_Loss: 28835.7495
Test_Loss: 29291.8639
--------
Epoch: 2/100
Train_Loss: 28662.1103
Test_Loss: 28882.9371
--------
Epoch: 3/100
Train_Loss: 28002.1978
Test_Loss: 28140.5881
--------
Epoch: 4/100
Train_Loss: 27111.3774
Test_Loss: 27058.8434
--------
Epoch: 5/100
Train_Loss: 25937.1342
Test_Loss: 25694.0805
--------
Epoch: 6/100
Train_Loss: 24398.0963
Test_Loss: 24091.6056
--------
Epoch: 7/100
Train_Loss: 22927.1522
Test_Loss: 22313.3902
--------
Epoch: 8/100
Train_Loss: 20866.5847
Test_Loss: 20410.4621
--------
Epoch: 9/100
Train_Loss: 19009.0562
Test_Loss: 18503.0079
--------
Epoch: 10/100
Train_Loss: 17130.0656
Test_Loss: 16618.2228
--------
Epoch: 11/100
Train_Loss: 15194.3707
Test_Loss: 14780.9455
--------
Epoch: 12/100
Train_Loss: 13420.5298
Test_Loss: 13043.0311
--------
Epoch: 13/100
Train_Loss: 11886.9439
Test_Loss: 11488.2336
--------
Epoch: 14/100
Train_Loss: 10344.0385
Test_Loss: 10077.1257
--------
Epoch: 15/100
Train_Loss: 9053.9670
Test_Loss: 8

In [ ]:
plt.figure()
plt.title('Train and Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, epoch + 1), train_loss_list, color='blue', linestyle='-', label='Train_Loss')
plt.plot(range(1, epoch + 1), test_loss_list, color='red', linestyle='--', label='Test_Loss')
plt.legend()

plt.show()